### MDSI Advanced Machine Learning Applications

**Student: Ivan Cheung**

**Assignment: AT2**


In [1]:
## load libraries
import pandas as pd

In [ ]:
## load data
df_train = pd.read_csv('../../data/raw/sales_train.csv')
df_items = pd.read_csv('../../data/raw/items_weekly_sell_prices.csv')
df_cal = pd.read_csv('../../data/raw/calendar.csv')
df_events = pd.read_csv('../../data/raw/calendar_events.csv')

### Exploratory Data Analysis:

1) Identify columns of interest

2) Identify how columns should be encoded for the model:
    - Categorical columns with large volume of unique values: Label Encoding
    - Categorical columns with low volume of unique values: One Hot Encoding
    - numerical columns: StandardScalar
    - Date column: no encoding

In [14]:
df_train.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1532,d_1533,d_1534,d_1535,d_1536,d_1537,d_1538,d_1539,d_1540,d_1541
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,1,0,1,0,1,0,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,8,2,0,8,2,3,1,1,3,8
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,0,1,3,2,1,1,2,2,3


In [15]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6841121 entries, 0 to 6841120
Data columns (total 4 columns):
 #   Column      Dtype  
---  ------      -----  
 0   store_id    object 
 1   item_id     object 
 2   wm_yr_wk    int64  
 3   sell_price  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 208.8+ MB


In [16]:
df_cal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      1969 non-null   object
 1   wm_yr_wk  1969 non-null   int64 
 2   d         1969 non-null   object
dtypes: int64(1), object(2)
memory usage: 46.3+ KB


In [17]:
df_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167 entries, 0 to 166
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        167 non-null    object
 1   event_name  167 non-null    object
 2   event_type  167 non-null    object
dtypes: object(3)
memory usage: 4.0+ KB


In [18]:
## remove variables to recover memory
del(df_cal)
del(df_events)
del(df_items)

#### Feature construction:

1) merge item sales with sale price to calculate revenue per observation in df_train
2) pivot out calendar events and append calendar events to df_train table
3) clean date columns

In [19]:
## 1: pop id, this data is not needed
df_train.pop('id')

0        HOBBIES_1_001_CA_1_evaluation
1        HOBBIES_1_002_CA_1_evaluation
2        HOBBIES_1_003_CA_1_evaluation
3        HOBBIES_1_004_CA_1_evaluation
4        HOBBIES_1_005_CA_1_evaluation
                     ...              
30485      FOODS_3_823_WI_3_evaluation
30486      FOODS_3_824_WI_3_evaluation
30487      FOODS_3_825_WI_3_evaluation
30488      FOODS_3_826_WI_3_evaluation
30489      FOODS_3_827_WI_3_evaluation
Name: id, Length: 30490, dtype: object

In [ ]:
## 2: apply melt to convert all date columns into a single column
df_train = pd.melt(df_train, id_vars=['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'd', value_name = 'items_sold')

In [21]:
def convert_date(d):
    diff = pd.to_datetime(d) - pd.to_datetime("2011-01-01")
    return diff

In [22]:
## 3. merge with calendar table to bring in wm_yr_wk col
df_cal = pd.read_csv('../../data/raw/calendar.csv')
df_cal['datediff'] = df_cal['date'].apply(convert_date).dt.days

In [23]:
df_cal

,date,wm_yr_wk,d,datediff
0,2011-01-29,11101,d_1,28
1,2011-01-30,11101,d_2,29
2,2011-01-31,11101,d_3,30
3,2011-02-01,11101,d_4,31
4,2011-02-02,11101,d_5,32
...,...,...,...,...
1964,2016-06-15,11620,d_1965,1992
1965,2016-06-16,11620,d_1966,1993
1966,2016-06-17,11620,d_1967,1994
1967,2016-06-18,11621,d_1968,1995


In [24]:
df_train = pd.merge(df_train, df_cal, on='d', how='left')
df_train.pop('d')
del(df_cal)
print('remove unneeded column and table to recover memory')

KeyboardInterrupt: 

In [ ]:
## 4. merge with items sale prices to calculate revenue
df_items = pd.read_csv('../../data/raw/items_weekly_sell_prices.csv')
df_train = pd.merge(df_train, df_items, left_on = ['store_id', 'item_id', 'wm_yr_wk'], right_on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')

df_train.pop('wm_yr_wk')
del(df_items)
print('remove unneeded column and table to recover memory')

remove unneeded column and table to recover memory


In [ ]:
## 5. calculate revenue and drop item sales and sell price columns
df_train['items_sold'] = df_train['items_sold'].fillna(0)
df_train['sell_price'] = df_train['sell_price'].fillna(0)

df_train['revenue'] = df_train['items_sold'] * df_train['sell_price']
df_train.pop('items_sold')
df_train.pop('sell_price')
print('remove unneeded column to recover memory')

remove unneeded column to recover memory


In [ ]:
df_train

,item_id,dept_id,cat_id,store_id,state_id,date,datediff,revenue
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,2011-01-29,28,0.00
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,2011-01-29,28,0.00
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,2011-01-29,28,0.00
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,2011-01-29,28,0.00
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,2011-01-29,28,0.00
...,...,...,...,...,...,...,...,...
46985085,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,2015-04-18,1568,5.76
46985086,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,2015-04-18,1568,0.00
46985087,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,2015-04-18,1568,0.00
46985088,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,2015-04-18,1568,0.00


In [ ]:
## 6. expand calendar events to count the number of holidays on a given date
df_events = pd.read_csv('../../data/raw/calendar_events.csv')

df_events = pd.pivot_table(df_events, index='date', values='event_name', columns='event_type', aggfunc='count', fill_value= 0)
df_events = df_events.rename_axis(columns= None).reset_index()

### rename cols for better readbility
df_events = df_events.rename(columns={"Cultural": "event_cultural", "National": "event_national", "Religious": "event_religious", "Sporting": "event_sport"})
df_events

,date,event_cultural,event_national,event_religious,event_sport
0,2011-02-06,0,0,0,1
1,2011-02-14,1,0,0,0
2,2011-02-21,0,1,0,0
3,2011-03-09,0,0,1,0
4,2011-03-16,0,0,1,0
...,...,...,...,...,...
157,2016-05-08,1,0,0,0
158,2016-05-30,0,1,0,0
159,2016-06-02,0,0,0,1
160,2016-06-07,0,0,1,0


In [ ]:
### 7. merge df_train with cal events
df_train = pd.merge(df_train, df_events, left_on = ['date'], right_on = ['date'], how = 'left')
df_train['event_cultural'] = df_train['event_cultural'].fillna(0).astype('int64')
df_train['event_national'] = df_train['event_national'].fillna(0).astype('int64')
df_train['event_religious'] = df_train['event_religious'].fillna(0).astype('int64')
df_train['event_sport'] = df_train['event_sport'].fillna(0).astype('int64')
del(df_events)

In [ ]:
df_train.pop('date')
print('remove column to recover memory')

remove column to recover memory


In [ ]:
df_train.head()

,item_id,dept_id,cat_id,store_id,state_id,datediff,revenue,event_cultural,event_national,event_religious,event_sport
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,28,0.0,0,0,0,0
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,28,0.0,0,0,0,0
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,28,0.0,0,0,0,0
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,28,0.0,0,0,0,0
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,28,0.0,0,0,0,0


In [ ]:
df_train.to_parquet('../../data/processed/df_train.parquet')

In [2]:
df_train = pd.read_parquet('../../data/processed/df_train.parquet')

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46985090 entries, 0 to 46985089
Data columns (total 11 columns):
 #   Column           Dtype  
---  ------           -----  
 0   item_id          object 
 1   dept_id          object 
 2   cat_id           object 
 3   store_id         object 
 4   state_id         object 
 5   datediff         int64  
 6   revenue          float64
 7   event_cultural   int64  
 8   event_national   int64  
 9   event_religious  int64  
 10  event_sport      int64  
dtypes: float64(1), int64(5), object(5)
memory usage: 3.9+ GB


In [3]:
## Swapping out column type to reduce memory usage
df_train['cat_id'] = df_train['cat_id'].astype("category")
df_train['dept_id'] = df_train['dept_id'].astype("category")
df_train['store_id'] = df_train['store_id'].astype("category")
df_train['state_id'] = df_train['state_id'].astype("category")
df_train['item_id'] = df_train['item_id'].astype("category")

In [4]:
## swapping out int for unsigned
df_train['datediff'] = pd.to_numeric(df_train['datediff'], downcast='unsigned')
df_train['event_cultural'] = pd.to_numeric(df_train['event_cultural'], downcast='unsigned')
df_train['event_national'] = pd.to_numeric(df_train['event_national'], downcast='unsigned')
df_train['event_religious'] = pd.to_numeric(df_train['event_religious'], downcast='unsigned')
df_train['event_sport'] = pd.to_numeric(df_train['event_sport'], downcast='unsigned')

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46985090 entries, 0 to 46985089
Data columns (total 11 columns):
 #   Column           Dtype   
---  ------           -----   
 0   item_id          category
 1   dept_id          category
 2   cat_id           category
 3   store_id         category
 4   state_id         category
 5   datediff         uint16  
 6   revenue          float64 
 7   event_cultural   uint8   
 8   event_national   uint8   
 9   event_religious  uint8   
 10  event_sport      uint8   
dtypes: category(5), float64(1), uint16(1), uint8(4)
memory usage: 896.3 MB


In [ ]:
print("df_train['item_id'] " + str(df_train['item_id'].nunique()))
print("df_train['dept_id'] " + str(df_train['dept_id'].nunique()))
print("df_train['cat_id'] " + str(df_train['cat_id'].nunique()))
print("df_train['store_id'] " + str(df_train['store_id'].nunique()))
print("df_train['state_id'] " + str(df_train['state_id'].nunique()))

df_train['item_id'] 3049
df_train['dept_id'] 7
df_train['cat_id'] 3
df_train['store_id'] 10
df_train['state_id'] 3


### Feature Engineering:

1) Build preprocessing steps

2) insert steps into preprocessor.

In [6]:
## define label encode cols, OHE cols and num cols
target = ['revenue']
label_cols = ['item_id']
cat_cols = ['dept_id', 'cat_id', 'state_id', 'store_id']
num_cols = ['datediff', 'event_cultural', 'event_national', 'event_religious', 'event_sport']

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor

In [8]:
# initiate transformer for num features
num_transformer = Pipeline(
    steps=[
        ('scaler', StandardScaler())
    ]
)

In [9]:
# initiate transformer for cat features
cat_transformer = Pipeline(
    steps=[
        ('one_hot_encoder', OneHotEncoder(sparse=False, drop='first'))
    ]
)

In [10]:
label_transformer = Pipeline(
    steps=[
        ('label_encoder', LabelEncoder())
    ]
)

In [11]:
ordinal_transformer = Pipeline(
    steps=[
        ('ordinal_encoder', OrdinalEncoder())
    ]
)

In [12]:
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num_cols', num_transformer, num_cols),
        ('cat_cols', cat_transformer, cat_cols),
        ('label_cols', ordinal_transformer, label_cols)
    ]
)

In [13]:
regression_pipe = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        #('linear', LinearRegression())
        ('sgd', SGDRegressor(penalty='elasticnet', random_state=42))
    ]
)

In [14]:
## splitting the dataset
from sklearn.model_selection import train_test_split
target = df_train.pop('revenue')
X_train, X_test, y_train, y_test = train_test_split(df_train, target, test_size=0.2, random_state=8)

In [15]:
## remove variables to recover memory
del(df_train)
del(target)

In [16]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37588072 entries, 41997901 to 21752169
Data columns (total 10 columns):
 #   Column           Dtype   
---  ------           -----   
 0   item_id          category
 1   dept_id          category
 2   cat_id           category
 3   store_id         category
 4   state_id         category
 5   datediff         uint16  
 6   event_cultural   uint8   
 7   event_national   uint8   
 8   event_religious  uint8   
 9   event_sport      uint8   
dtypes: category(5), uint16(1), uint8(4)
memory usage: 717.0 MB


In [20]:
X_train.to_csv('../../data/interim/X_train.csv', index=False)
X_test.to_csv('../../data/interim/X_test.csv', index=False)
y_train.to_csv('../../data/interim/y_train.csv', index=False)
y_test.to_csv('../../data/interim/y_test.csv', index=False)

In [40]:
### load
X_train = pd.read_csv('../../data/interim/X_train.csv')
y_train = pd.read_csv('../../data/interim/y_train.csv')

In [41]:
### load
X_test = pd.read_csv('../../data/interim/X_test.csv')
y_test = pd.read_csv('../../data/interim/y_test.csv')

In [17]:
regression_pipe.fit(X_train, y_train)

c:\Users\Ivan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\Ivan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num_cols',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['datediff', 'event_cultural',
                                                   'event_national',
                                                   'event_religious',
                                                   'event_sport']),
                                                 ('cat_cols',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 sparse=False))]),
                                                  ['dept_id', 'cat_id',
                                                   'state_id', 'store_id']),
                                                 ('label_cols',
                                                  Pipeline(steps=[('ordinal_encoder',
                                                                   OrdinalEncoder())]),
                                                  ['item_id'])])),
                ('sgd', SGDRegressor(penalty='elasticnet', random_state=42))])

In [19]:
## recover memory
del(X_train)
del(y_train)

In [18]:
# Save model
from joblib import dump
dump(regression_pipe, '../../models/predictive/regression.joblib')

['../../models/predictive/regression.joblib']

In [20]:
y_pred = regression_pipe.predict(X_test)

In [21]:
## RMSE evaluation
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(y_test, y_pred, squared=False)
rmse

83321078069599.31

In [22]:
# r2 score
from sklearn.metrics import r2_score

r2_score(y_test, y_pred)

-8.265622837106016e+25